# Privacy Audit - DPO Training (Stage 2)

使用 DPO 对 SFT 模型进行偏好优化，观察隐私风险在 preference optimization 阶段的变化。

**运行方式**: 通过本地 IDE Colab 插件连接 Colab Runtime

## 1. 安装依赖

In [ ]:
!pip install -q datasets transformers peft trl accelerate

## 2. 检查 GPU

In [ ]:
import torch
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")

## 3. 上传本地文件到 Colab Runtime

需要上传:
- `data/preference_data.jsonl` - 偏好数据
- `models/stage1_sft/` - SFT 模型 (或从 HuggingFace 加载)

In [ ]:
from google.colab import files
import os

# 创建目录
os.makedirs("data", exist_ok=True)
os.makedirs("models/stage1_sft", exist_ok=True)
os.makedirs("models/stage2_dpo", exist_ok=True)

print("请上传 preference_data.jsonl 文件:")
uploaded = files.upload()

# 移动到正确位置
for filename in uploaded.keys():
    if filename.endswith('.jsonl'):
        os.rename(filename, f"data/{filename}")
        print(f"✅ {filename} -> data/{filename}")

In [ ]:
# 上传 SFT 模型文件 (adapter)
print("请上传 SFT 模型文件 (adapter_config.json, adapter_model.safetensors 等):")
uploaded_model = files.upload()

for filename in uploaded_model.keys():
    os.rename(filename, f"models/stage1_sft/{filename}")
    print(f"✅ {filename} -> models/stage1_sft/{filename}")

print("\nSFT 模型目录内容:")
print(os.listdir("models/stage1_sft"))

## 4. 配置路径

In [ ]:
# 配置
BASE_MODEL_NAME = "Qwen/Qwen2.5-0.5B-Instruct"
SFT_MODEL_DIR = "models/stage1_sft"
PREFERENCE_DATA = "data/preference_data.jsonl"
OUTPUT_DIR = "models/stage2_dpo"

print(f"Base model: {BASE_MODEL_NAME}")
print(f"SFT model: {SFT_MODEL_DIR}")
print(f"Preference data: {PREFERENCE_DATA}")
print(f"Output: {OUTPUT_DIR}")

# 验证文件存在
assert os.path.exists(PREFERENCE_DATA), f"❌ {PREFERENCE_DATA} not found"
print(f"\n✅ Preference data exists: {os.path.getsize(PREFERENCE_DATA)} bytes")

## 5. 加载模型和数据

In [ ]:
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import PeftModel
from trl import DPOTrainer, DPOConfig

# 加载 tokenizer
print("[INFO] Loading tokenizer...")
tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL_NAME)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
print(f"[OK] Tokenizer loaded. Vocab size: {len(tokenizer)}")

# 加载 base model
print("[INFO] Loading base model...")
base_model = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL_NAME,
    device_map="auto",
    torch_dtype=torch.bfloat16
)
print("[OK] Base model loaded!")

# 加载 SFT adapter (如果有)
if os.path.exists(f"{SFT_MODEL_DIR}/adapter_config.json"):
    print("[INFO] Loading SFT adapter (Stage 1)...")
    model = PeftModel.from_pretrained(base_model, SFT_MODEL_DIR)
    print("[OK] SFT adapter loaded!")
else:
    print("[INFO] No SFT adapter found, using base model directly")
    model = base_model

In [ ]:
# 加载偏好数据
print("[INFO] Loading preference dataset...")
dataset = load_dataset("json", data_files=PREFERENCE_DATA, split="train")
print(f"[OK] Dataset loaded. Number of examples: {len(dataset)}")
print(f"[INFO] Sample: {dataset[0]}")

## 6. 配置 DPO Trainer

In [ ]:
print("[INFO] Configuring DPO Trainer...")

dpo_config = DPOConfig(
    learning_rate=5e-5,
    num_train_epochs=1,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=4,
    output_dir=OUTPUT_DIR,
    logging_steps=10,
    save_steps=100,
    beta=0.1,
    max_length=512,
    max_prompt_length=256,
    bf16=True,
)

trainer = DPOTrainer(
    model=model,
    ref_model=None,
    args=dpo_config,
    train_dataset=dataset,
    processing_class=tokenizer,
)
print("[OK] DPO Trainer initialized!")

## 7. 开始训练

In [ ]:
print("=" * 60)
print("[INFO] Starting DPO training (Stage 2)...")
print("=" * 60)
trainer.train()

## 8. 保存模型

In [ ]:
print("[INFO] Saving DPO model...")
trainer.save_model(OUTPUT_DIR)
tokenizer.save_pretrained(OUTPUT_DIR)
print(f"[DONE] DPO model saved to {OUTPUT_DIR}")
print(f"\nContents: {os.listdir(OUTPUT_DIR)}")

## 9. 下载训练好的模型到本地

In [ ]:
# 打包并下载
import shutil

print("[INFO] Packaging model for download...")
shutil.make_archive("stage2_dpo", 'zip', OUTPUT_DIR)
print("[OK] Created stage2_dpo.zip")

print("\n[INFO] Downloading model...")
files.download("stage2_dpo.zip")
print("[DONE] Download started!")